# Project #1 -  Information Retrieval


Cory Randolph

9/10/2022

CMPE 297


# Load Data

Download the data from the CMU [link](http://www.cs.cmu.edu/~ark/QA-data/).

In [1]:
!wget http://www.cs.cmu.edu/~ark/QA-data/data/Question_Answer_Dataset_v1.2.tar.gz

zsh:1: command not found: wget


Extract the contents of the zipped file

In [2]:
# Define the file name and run the unzip command
data_file_name = "Question_Answer_Dataset_v1.2.tar.gz"
!sudo tar -xvf $data_file_name

# Clear output for this cell
from IPython.display import clear_output
clear_output()

# Display success message if unziped folder exists
from pathlib import Path
data_file_path = Path("/content/Question_Answer_Dataset_v1.2")
if data_file_path.is_dir():
  print("Successfully Unzipped CMU Data")
else:
  print("Something went wrong and the CMU data was not unzipped")

Something went wrong and the CMU data was not unzipped


# Clean Data

## Question Answer Pair Data

First load in the data as a pandas dataframe for the questions and answers.


In [3]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df_qa_pairs_s08 = pd.read_csv("/content/Question_Answer_Dataset_v1.2/S08/question_answer_pairs.txt", sep="\t", encoding= "ISO-8859-1")
df_qa_pairs_s09 = pd.read_csv("/content/Question_Answer_Dataset_v1.2/S09/question_answer_pairs.txt", sep="\t", encoding= "ISO-8859-1")
df_qa_pairs_s10 = pd.read_csv("/content/Question_Answer_Dataset_v1.2/S10/question_answer_pairs.txt", sep="\t", encoding= "ISO-8859-1")

Add a column to each question and answer pair dataframe so indicate which dataset it came from (i.e. S08, S09, S10).

In [ ]:
df_qa_pairs_s08.insert(0, 'Dataset', "S08")
df_qa_pairs_s09.insert(0, 'Dataset', "S09")
df_qa_pairs_s10.insert(0, 'Dataset', "S10")

In [ ]:
df_qa_pairs_s08.head(3)

,Dataset,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,S08,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,data/set3/a4
1,S08,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,easy,easy,data/set3/a4
2,S08,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,easy,medium,data/set3/a4


Combine all of them into one dataframe

In [ ]:
df_qa_pairs = df_qa_pairs_s08.append([df_qa_pairs_s09, df_qa_pairs_s10])
df_qa_pairs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3998 entries, 0 to 1457
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Dataset                   3998 non-null   object
 1   ArticleTitle              3998 non-null   object
 2   Question                  3961 non-null   object
 3   Answer                    3422 non-null   object
 4   DifficultyFromQuestioner  3043 non-null   object
 5   DifficultyFromAnswerer    3418 non-null   object
 6   ArticleFile               3996 non-null   object
dtypes: object(7)
memory usage: 249.9+ KB


Clean up this data by removing rows that have missing Questions, Answers, or Article Files

In [ ]:
df_qa_pairs = df_qa_pairs.dropna(subset=['Question', 'Answer', 'ArticleFile'])

Next drow rows where the Question, Answer, and ArticleFile are the same values.

In [ ]:
df_qa_pairs = df_qa_pairs.drop_duplicates(subset=['Question', 'Answer', 'ArticleFile'], keep='first')

Create a modified ArticleFile column that adds in the extra path information from these local Colab files so that they can be matched from the custom information retrieval system.

In [ ]:
df_qa_pairs["ArticleFileCustom"] = df_qa_pairs.apply(lambda row: f"/content/Question_Answer_Dataset_v1.2/{row['Dataset']}/{row['ArticleFile']}.txt", axis =1)

In [ ]:
df_qa_pairs[["ArticleFile","ArticleFileCustom"]].head(3)

,ArticleFile,ArticleFileCustom
0,data/set3/a4,/content/Question_Answer_Dataset_v1.2/S08/data...
1,data/set3/a4,/content/Question_Answer_Dataset_v1.2/S08/data...
2,data/set3/a4,/content/Question_Answer_Dataset_v1.2/S08/data...


Combine the rows where the question is the same but the referenced article is different between the two answers

In [ ]:
df_qa_pairs.columns

Index(['Dataset', 'ArticleTitle', 'Question', 'Answer',
       'DifficultyFromQuestioner', 'DifficultyFromAnswerer', 'ArticleFile',
       'ArticleFileCustom'],
      dtype='object')

In [ ]:
def create_combined_article_files(df_qa_pairs: pd.DataFrame):
  # Find any questions that have multiple files with the correct answer and
  # combine them into a new column with a comma separating them
  df_temp_01 = df_qa_pairs.copy()
  df_temp_01 = df_temp_01[["Question", "ArticleFileCustom"]]
  df_temp_01 = df_temp_01.drop_duplicates(subset=["Question", "ArticleFileCustom"])
  df_temp_01["ArticleFilesCustom"] = df_temp_01[["Question", "ArticleFileCustom"]].groupby(["Question"])["ArticleFileCustom"].transform(lambda x: ",".join(x))
  df_temp_01 = df_temp_01[["Question", "ArticleFilesCustom"]].drop_duplicates()

  # Add the combined column as a new column
  df_temp_02 = df_qa_pairs.copy()
  df_temp_02 = df_temp_02.merge(df_temp_01, how="left", on = "Question")
  df_temp_02 = df_temp_02.drop(["ArticleFileCustom"], axis=1)
  
  return df_temp_02

In [ ]:
df_qa_pairs = create_combined_article_files(df_qa_pairs = df_qa_pairs)

Show the summary infor after cleaning

In [ ]:
df_qa_pairs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3111 entries, 0 to 3110
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Dataset                   3111 non-null   object
 1   ArticleTitle              3111 non-null   object
 2   Question                  3111 non-null   object
 3   Answer                    3111 non-null   object
 4   DifficultyFromQuestioner  2429 non-null   object
 5   DifficultyFromAnswerer    3108 non-null   object
 6   ArticleFile               3111 non-null   object
 7   ArticleFilesCustom        3111 non-null   object
dtypes: object(8)
memory usage: 218.7+ KB


# TF-IDF

Implement TF-IDF for the various datasets in order to find the most appropriate documents based on a given query.

In [ ]:
import numpy as np
import pandas as pd
import os

## Create Corpus
Create a doctionary that holds the corpus and the file name and the text of the document.

In [ ]:
def create_document_paths() -> list:
  from pathlib import Path
  
  # Root directory name
  dirname = '/content/Question_Answer_Dataset_v1.2'
  
  # Provide directory name to Path() function and pattern for finding 
  # the answer files that match "a" and end in ".txt"
  paths = Path(dirname).glob('**/a*.txt',)
  
  # Iterate over all answer files
  document_paths = []
  for path in paths:
    document_paths.append(str(path))
    # print(path)  # Print file name

  # Sort the paths for repeatability 
  document_paths = sorted(document_paths)

  return document_paths

In [ ]:
document_paths = create_document_paths()

Display the total number of documents in this corpus (i.e. the number of document paths we have

In [ ]:
len(document_paths)

150

Display 5 sample documents paths

In [ ]:
document_paths_sample = ['/content/Question_Answer_Dataset_v1.2/S08/data/set4/a1.txt',
  '/content/Question_Answer_Dataset_v1.2/S08/data/set4/a2.txt',
  '/content/Question_Answer_Dataset_v1.2/S08/data/set4/a3.txt',
  '/content/Question_Answer_Dataset_v1.2/S08/data/set4/a4.txt',
  '/content/Question_Answer_Dataset_v1.2/S08/data/set4/a5.txt'
 ]

document_paths_sample

#  First 5 unnamed docs:
# document_paths_sample = document_paths[:5]
# document_paths_sample 

['/content/Question_Answer_Dataset_v1.2/S08/data/set4/a1.txt',
 '/content/Question_Answer_Dataset_v1.2/S08/data/set4/a2.txt',
 '/content/Question_Answer_Dataset_v1.2/S08/data/set4/a3.txt',
 '/content/Question_Answer_Dataset_v1.2/S08/data/set4/a4.txt',
 '/content/Question_Answer_Dataset_v1.2/S08/data/set4/a5.txt']

Create a corpus of all the documents

In [ ]:
import io

In [ ]:
def create_corpus(document_paths: list):
  corpus = {}

  """Limiting to the first 5 for initial testing, Change to all documents later"""
  for document_path in document_paths:
    with io.open(f'{document_path}','r', encoding="latin-1") as document:
      corpus[document_path] = document.read()

  return corpus

In [ ]:
corpus_sample = create_corpus(document_paths = document_paths_sample)

Display an example of a document path and document text form the corpus

In [ ]:
# Keys in the corpus are the files paths
corpus_sample.keys()

dict_keys(['/content/Question_Answer_Dataset_v1.2/S08/data/set4/a1.txt', '/content/Question_Answer_Dataset_v1.2/S08/data/set4/a2.txt', '/content/Question_Answer_Dataset_v1.2/S08/data/set4/a3.txt', '/content/Question_Answer_Dataset_v1.2/S08/data/set4/a4.txt', '/content/Question_Answer_Dataset_v1.2/S08/data/set4/a5.txt'])

In [ ]:
# Example of the first 300 characters of one of the documents
first_doc_path_sample :str = list(corpus_sample.keys())[0]
first_doc_path_sample
# corpus_sample[first_doc_path_sample][:300]

'/content/Question_Answer_Dataset_v1.2/S08/data/set4/a1.txt'

Create a single sample document that can be used for testing and development

In [ ]:
# Create a sample document that is just a single string
document_sample :str = corpus_sample[first_doc_path_sample]
# document_sample # Uncomment to display the full text of document

## Tokenize Document

Create a custom tokenizer that will take the document as input and then break it up into a list of tokens

In [ ]:
import numpy as np
import re

In [ ]:
def custom_tokenizer(document: str) -> list:
  """
  Create a custom tokenizer function. This should allow us to modify
  the tokenizer for experiments while keeping the rest of the processing
  pipeline the same.

  Note: Using Numpy arrays for faster performance over python lists
  """
  # Clean document (Note: We may want to keep the periods or new lines for forming sentences later)
  document = document.lower() # Lower case all letters
  document = re.sub(r'[^a-z0-9\s]',' ',document) # Remove all non letter, non-number and non space characters
  document = re.sub(r'\n',' ',document) # Remove line breaks
  document = re.sub(r'\s+',' ',document) # Remove Extra spaces


  # Split the sentence into words based on the space
  tokens = document.split()

  # Make sure each token doesn't have extra spaces on the ends
  tokens = [token.strip() for token in tokens]

  # Drop Stop words:
  # TODO:

  # Drop single letters (loose "I" and "a", but those are potentially stop word anyways)
  tokens = [token for token in tokens if len(token) > 1]

  # Turn the list into a numpy array for performance
  tokens = np.array(tokens)

  # Maybe add limitation to make sure the words are english characters/words (in nltk this is .alpha())
  # TODO:

  return tokens

In [ ]:
document_sample[:300]

'Isaac Newton\n\n\n\nSir Isaac Newton FRS ( ) (4 January 1643 â\x80\x93 March 31 1727)   [  OS: December 25 1642 â\x80\x93 March 20 1727 ]    was an English physicist, mathematician, astronomer, natural philosopher, and alchemist. His treatise PhilosophiÃ¦ Naturalis Principia Mathematica, published in 1687, describe'

Apply the custom tokenizer to the sample document

In [ ]:
custom_tokenizer(document_sample)[:20]

array(['isaac', 'newton', 'sir', 'isaac', 'newton', 'frs', 'january',
       '1643', 'march', '31', '1727', 'os', 'december', '25', '1642',
       'march', '20', '1727', 'was', 'an'], dtype='<U15')

Apply the tokenizer to the corpus. This will convert the string documents into lists of tokens/words in the document

In [ ]:
def tokenize_corpus(corpus: dict, tokenizer):
  corpus_tokenized = {}
  # Itterate through each document in the corpus and apply the custom tokenizer
  for document_path, document in corpus.items():
    # corpus[document_path] = [token for token in custom_tokenizer(corpus[document_path])]
    corpus_tokenized[document_path] = custom_tokenizer(document)

  return corpus_tokenized

In [ ]:
corpus_tokenized_sample = tokenize_corpus(corpus = corpus_sample, tokenizer = custom_tokenizer)

Create a single sample of a corpus document with the file path and document in it's tokenized form

In [ ]:
# Note the corpus_document_sample is a dictionary with one item in the form of {file_path: document as a list}
# document_tokenized_sample :dict = {first_doc_path_sample:corpus_tokenized_sample[first_doc_path_sample]}

# Create a sample document that is a list of the tokens form one document
document_tokenized_sample = corpus_tokenized_sample[first_doc_path_sample]
document_tokenized_sample[:15]

array(['isaac', 'newton', 'sir', 'isaac', 'newton', 'frs', 'january',
       '1643', 'march', '31', '1727', 'os', 'december', '25', '1642'],
      dtype='<U15')

## Create Vocabulary

Iterate through all of the documents and create a vocabulary on unique words found.

In [ ]:
def create_vocabulary(corpus : dict):
  # Add every word in the corpus to a set of vocabualry words
  vocabulary = np.empty(0)
  for document in corpus.values():
    # Add Each document to the vocabulary
    vocabulary = np.append(vocabulary,document)

  # Get just the unique values
  vocabulary = np.unique(vocabulary)

  # Convert the set into a sorted list
  vocabulary = np.sort(vocabulary)

  return vocabulary

In [ ]:
vocabulary_sample = create_vocabulary(corpus_tokenized_sample)

In [ ]:
len(vocabulary_sample)

5555

Display the first few words in the vocabulary

In [ ]:
vocabulary_sample[:20]

array(['000', '00189', '0028', '008', '01', '04', '045', '05',
       '0688168949', '0743215362', '0760710058', '082', '08817',
       '0914732334', '10', '100', '100rsd', '101', '103', '104'],
      dtype='<U32')

Note: After looking through this there are many additional ways to clean up these tokens such as removing white space, non letter characters, non-english characters, etc.

# Try breasking the calclualtions up to speed up

## Calculate Term Frequency (TF)

Iterate over all of the document_paths in the corpus and calculate the frequency that the terms appea in each document

In [ ]:
def calculate_term_frequency(corpus: dict, vocabulary: set):
  # Create a term frequency for every document in the corpus
  tf = {}

  for document_path in corpus:
    tf[document_path] = {word: np.count_nonzero(corpus[document_path] == word) / len(corpus[document_path]) for word in vocabulary}
    # tf[document_path] = {word: corpus[document_path].count(word) for word in vocabulary}

  return tf


In [ ]:
tf_sample = calculate_term_frequency(corpus = corpus_tokenized_sample, vocabulary = vocabulary_sample)

Display the first few example of the term frequency

In [ ]:
list(tf_sample[first_doc_path_sample].items())[:15]

[('000', 0.0),
 ('00189', 0.00015997440409534473),
 ('0028', 0.0),
 ('008', 0.0),
 ('01', 0.0),
 ('04', 0.0),
 ('045', 0.0),
 ('05', 0.0),
 ('0688168949', 0.0),
 ('0743215362', 0.0),
 ('0760710058', 0.0),
 ('082', 0.0),
 ('08817', 0.00015997440409534473),
 ('0914732334', 0.0),
 ('10', 0.0)]

## Calculate Inverse Document Frequency (IDF)

Iterate through all of the words in the vocabulary and calculate how many documents it's in from the corpus

In [ ]:
def calculate_inverse_document_frequency(corpus :dict, vocabulary: np.array):
  # For every word in the vocabulay calcualte the inverse frequency
  idf = {}

  for word in vocabulary:
    # freq = np.sum(word in corpus[document_path] for document_path in corpus)
    freq = (sum([1 for doc in corpus.values() if word in doc]) + 1)
    idf[word] = np.log10(len(corpus) / freq)

  return idf

In [ ]:
idf_sample = calculate_inverse_document_frequency(corpus = corpus_tokenized_sample, vocabulary = vocabulary_sample)

Display the first few examples of the Inverse Document Frequency

In [ ]:
list(idf_sample.items())[1000:1015]

[('cancer', 0.3979400086720376),
 ('cannon', 0.3979400086720376),
 ('cannonball', 0.3979400086720376),
 ('cannot', 0.22184874961635637),
 ('capable', 0.3979400086720376),
 ('caparica', 0.3979400086720376),
 ('capital', 0.09691001300805642),
 ('captain', 0.3979400086720376),
 ('caption', 0.3979400086720376),
 ('car', 0.3979400086720376),
 ('cardinal', 0.3979400086720376),
 ('care', 0.22184874961635637),
 ('careful', 0.3979400086720376),
 ('carefully', 0.3979400086720376),
 ('carlson', 0.3979400086720376)]

## Calculate Term Frequence-Inverse Document Frequency (TF-IDF)

Iterate through every document in the corpus and then multiply the Term Frequency (TF) with the Inter Document Frequency (IDF). Store this in a dictionary with the document path as the key and the vocabulary-base vector .

In [ ]:
def calculate_tfidf(tf: dict, idf: dict):
  tfidf = {}

  # Since the tf and idf are already in order and based on the vocabulary
  # we can convert them to numpy arrays and multiple
  idf_array = np.array(list(idf.values()))

  for document_path in tf:
    # Convert the tf values into a numpy array
    tf_array = np.array(list(tf[document_path].values()))

    tfidf[document_path] = np.multiply(tf_array,idf_array)

  return tfidf

In [ ]:
tfidf_sample = calculate_tfidf( tf = tf_sample, idf = idf_sample)

Display Sample

In [ ]:
tfidf_sample[first_doc_path_sample]

array([0.00000000e+00, 6.36602158e-05, 0.00000000e+00, ...,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

## Calculate Term Frequence-Inverse Document Frequency (TF-IDF)

Iterate through every document in the corpus and then multiply the Term Frequency (TF) with the Inter Document Frequency (IDF). Store this in a dictionary with the document path as the key and the vocabulary-base vector .

In [ ]:
# def calculate_tfidf(corpus:dict, document: np.array, word: str ):
#   # Calculate TF
#   # tf = document.count(word) /len(document)
#   tf = np.count_nonzero(document == word) /len(document)
#   # tf = (document == word).sum() /len(document) 

#   # Inverse Document Frequency
#   idf = np.log10(len(corpus.keys()) / (sum([1 for doc in corpus.values() if word in doc]) + 1))

#   # TF-IDF
#   tfidf = round(tf*idf,4)
  
#   return tfidf

In [ ]:
# calculate_tfidf(corpus = corpus_tokenized_sample, document = document_tokenized_sample, word = 'isaac')

0.0015

Create vector from vocabulary for each document

In [ ]:
# def tfidf_for_corpus(corpus: dict, vocabulary: list, print_progress: bool = False):
#   tfidf = {}
#   document_counter = 0

#   # Itterate through each document in the corpus
#   for document_path, document in corpus.items():
#     # Display progress option for every 10 documents
#     if print_progress:
#       document_counter =+ 1
#       if document_counter % 10 == 0:
#         print(f"Calculating TFIDF for {document_path}")

#     # Create empty list
#     cur_document_tfidf = np.zeros(len(vocabulary))

#     # Itterate through each word in the vocabulary and calculate TFIDF
#     # for word in vocabulary:
#     for idx, word in enumerate(vocabulary):
#       cur_document_tfidf[idx] = calculate_tfidf(corpus = corpus, document = document, word = word)

#     tfidf[document_path] = cur_document_tfidf

#   return tfidf

In [ ]:
# tfidf_sample = tfidf_for_corpus(corpus = corpus_tokenized_sample, vocabulary = vocabulary_sample, print_progress=True)

## Query Based on TF_IDF

Take an input query and use the generated TF-IDF results to provide a ranked list of matching documents with potential answer to the query

In [ ]:
query_sample = "Is it true that newton saw god as the master creator"

Apply the custom tokenizer to the input query

In [ ]:
query_tokenized_sample = custom_tokenizer(query_sample)
query_tokenized_sample

array(['is', 'it', 'true', 'that', 'newton', 'saw', 'god', 'as', 'the',
       'master', 'creator'], dtype='<U7')

In [ ]:
tfidf_sample[first_doc_path_sample][:10]

array([0.00000000e+00, 6.36602158e-05, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00])

### Cosine Simlarity

Apply cosine similarity between the query and each document. Since the tfidf dictionary is already vectorized based on the vocabulary we just need to vectorize the input query.

In [ ]:
def vectorize_query(query: str, vocabulary :list, tokenizer: object):
  # Tokenize query with custom tokenizer
  query_tokenized = tokenizer(query)

  # Create a vector of 0's with the vocabulary lenght
  query_vector = np.zeros(len(vocabulary))

  # Itterate through each word in the vocabulary list
  for index, vocab_word in enumerate(vocabulary):
    # Set the vector value equal to the tfidf value when ever the words match
    for query_word in query_tokenized:
      if query_word == vocab_word:
        query_vector[index] = 1
      else:
        pass

  return query_vector

Show example vectorized query

In [ ]:
query_vectorized_sample = vectorize_query(query = query_sample, vocabulary = vocabulary_sample, tokenizer = custom_tokenizer)
query_vectorized_sample[:10]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

Define Cosine simliarity formula

In [ ]:
def cosine_sim(vector_a, vector_b):
    cosine_sim = np.dot(vector_a, vector_b)/(np.linalg.norm(vector_a)*np.linalg.norm(vector_b))
    return cosine_sim

Example Calculation

In [ ]:
cosine_sim(query_vectorized_sample, tfidf_sample[first_doc_path_sample])

0.16481043105913037

Combine this all to gether to calculate the cosine similarity for all queries and output the ranked document paths

In [ ]:
def calculate_query_cosine_similarity_for_all_docs(query: str, tfidf: dict, vocabulary : list, tokenizer: object, n: int):
  """
  Apply the cosine similarity measure for all documents and return the top N results

  Input:
    n = Top N documents for a given query
  """
  # Vectorize query
  query_vectorized = vectorize_query(query, vocabulary, tokenizer)

  # Generate score 
  cosine_similarity_score = []
  for document_path, tfidf_vector in tfidf.items():
    # Calculate the query similarity
    cosine_similarity_score.append([document_path, cosine_sim(query_vectorized, tfidf_vector)])

  # Sort scores
  cosine_similarity_score_sorted = sorted(cosine_similarity_score, key=lambda x: x[1], reverse=True)

  # Return just the top n resutls
  top_n_document_paths = [document_path[0] for document_path in cosine_similarity_score_sorted]

  # Format the returned results into a comma separated string
  top_n_document_paths = ",".join(top_n_document_paths)

  return top_n_document_paths

In [ ]:
calculate_query_cosine_similarity_for_all_docs(query = query_sample, tfidf = tfidf_sample, vocabulary = vocabulary_sample, tokenizer = custom_tokenizer, n = 5)

'/content/Question_Answer_Dataset_v1.2/S08/data/set4/a1.txt,/content/Question_Answer_Dataset_v1.2/S08/data/set4/a5.txt,/content/Question_Answer_Dataset_v1.2/S08/data/set4/a3.txt,/content/Question_Answer_Dataset_v1.2/S08/data/set4/a2.txt,/content/Question_Answer_Dataset_v1.2/S08/data/set4/a4.txt'

Create a final data structure with the document path and the ranked documents

In [ ]:
example_queries = [
    "Is it true that newton saw god as the master creator",
    "Who was born precisely at midnight during an electrical storm , to a Serbian family in the village of Smiljan near Gospić",
    "what was Blaise Pascal famos for",
]

In [ ]:
df_cosine_similarity = pd.DataFrame(example_queries, columns = ["query"]) #Substitute actual questiond from docs later

Add the top n resulting documents that are similar to the query

In [ ]:
df_cosine_similarity["matching_documents"] = df_cosine_similarity["query"].apply(lambda query: calculate_query_cosine_similarity_for_all_docs(query = query, tfidf = tfidf_sample, vocabulary = vocabulary_sample, tokenizer = custom_tokenizer, n = 5))

In [ ]:
df_cosine_similarity.head()

,query,matching_documents
0,Is it true that newton saw god as the master c...,/content/Question_Answer_Dataset_v1.2/S08/data...
1,Who was born precisely at midnight during an e...,/content/Question_Answer_Dataset_v1.2/S08/data...
2,what was Blaise Pascal famos for,/content/Question_Answer_Dataset_v1.2/S08/data...


## Save Intermittent Files

Create a way to save smaller objects as pickle files during the development process, since some functions take ~20 mintures to run on thw whole corpus this will make development time shorter.

In [ ]:
import pickle

In [ ]:
def save_as_pickle(object_to_be_saved, file_name):
  with open(file_name, 'wb') as handle:
      pickle.dump(object_to_be_saved, handle, protocol=pickle.HIGHEST_PROTOCOL)
  print(f"Pickle file has been saved to {file_name}")
  return None

def read_pickle(file_name):
  with open(file_name, 'rb') as handle:
      object_to_be_read = pickle.load(handle)
  return object_to_be_read

In [ ]:
# save_as_pickle(corpus, "corpus_item.pickle")

Add a current time stamp function for file naming

In [ ]:
import datetime

In [ ]:
def current_time_stamp():
  ts = datetime.datetime.now()
  # Convert to string format
  ts = ts.strftime("%y%m%d_%H_%M_%S")
  return ts

In [ ]:
current_time_stamp()

'221005_21_46_12'

## Combine to Single Function

Use all of the above functions combined into one repeateable function for experimenting with different configurations.

In [ ]:
def custom_information_retrieval():

  # Get all docuemnt paths from CMU QA Data sets
  print(f"Creating corpus...")
  document_paths = create_document_paths() #[:5]

  # Generate the corpus as a dictionary (note documents are still strings at this point)
  corpus = create_corpus(document_paths = document_paths)

  # Tokenize documents in corpus
  corpus_tokenized = tokenize_corpus(corpus = corpus, tokenizer = custom_tokenizer)
  save_as_pickle(object_to_be_saved = corpus_tokenized, file_name = f"corpus_tokenizedtemp.pickle")

  # Create vocabulary from corpus
  print(f"Creating vocabulary...")
  vocabulary = create_vocabulary(corpus = corpus_tokenized)
  save_as_pickle(object_to_be_saved = vocabulary, file_name = f"vocabulary_temp.pickle")

  # Calculate TF for every document in the corpus
  print(f"Calculating TF for corpus...")
  tf = calculate_term_frequency(corpus = corpus_tokenized, vocabulary = vocabulary)

  # Calculate IDF for every document in the corpus
  print(f"Calculating IDF for corpus...")
  idf = calculate_inverse_document_frequency(corpus = corpus_tokenized, vocabulary = vocabulary)

  # Calculate the tf-idf for every document in the corpus
  print(f"Calculating TF-IDF for corpus...")
  tfidf = calculate_tfidf( tf = tf, idf = idf)
  save_as_pickle(object_to_be_saved = tfidf, file_name = f"tfidf_temp.pickle")

  # Collect and vectorize all of the input queries
  # Columns e.g.  'Question', 'Answer', 'ArticleFileCustom'
  print(f"Calculating Cosine Similarity for all queries accross the corpus...")
  df_queries = df_qa_pairs.copy()
  df_queries['MatchingDocuments'] = df_queries["Question"].apply(lambda query: calculate_query_cosine_similarity_for_all_docs(query = query, tfidf = tfidf, vocabulary = vocabulary, tokenizer = custom_tokenizer, n = 5))

  return df_queries

In [ ]:
df_tfidf = custom_information_retrieval()

Creating corpus...
Pickle file has been saved to corpus_tokenizedtemp.pickle
Creating vocabulary...
Pickle file has been saved to vocabulary_temp.pickle
Calculating TF for corpus...
Calculating IDF for corpus...
Calculating TF-IDF for corpus...
Pickle file has been saved to tfidf_temp.pickle
Calculating Cosine Similarity for all queries accross the corpus...


In [ ]:
df_tfidf.head(3)
# df_tfidf[["ArticleTitle", "Question", "ArticleFileCustom", "MatchingDocuments"]]

,Dataset,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile,ArticleFilesCustom,MatchingDocuments
0,S08,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,data/set3/a4,/content/Question_Answer_Dataset_v1.2/S08/data...,/content/Question_Answer_Dataset_v1.2/S08/data...
1,S08,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,easy,easy,data/set3/a4,/content/Question_Answer_Dataset_v1.2/S08/data...,/content/Question_Answer_Dataset_v1.2/S08/data...
2,S08,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,easy,medium,data/set3/a4,/content/Question_Answer_Dataset_v1.2/S08/data...,/content/Question_Answer_Dataset_v1.2/S08/data...


Save the results localy for reloading in the future

In [ ]:
df_tfidf.to_csv("221003_df_tfidf.csv")

In [ ]:
df_tfidf.to_pickle('221003_df_tfidf.pickle')

In [ ]:
from google.colab import files
files.download('/content/221003_df_tfidf.pickle')
files.download('221003_df_tfidf.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Create Scoring Metric

## Validate QA Paths

First validate that the custom article paths created from the answers pair files actually match what what is in the corpus.

In [ ]:
example_doc_path_from_qa = df_qa_pairs["ArticleFilesCustom"].iloc[0]
example_doc_path_from_qa

'/content/Question_Answer_Dataset_v1.2/S08/data/set3/a4.txt'

In [ ]:
if example_doc_path_from_qa in document_paths:
  print(f"Document: {example_doc_path_from_qa} \n   Found in corpus")
else:
  print(f"Document {example_doc_path_from_qa} \n   Not Found in corpus")

Document: /content/Question_Answer_Dataset_v1.2/S08/data/set3/a4.txt 
   Found in corpus


Turn this logic into a function and appy to check all the files

In [ ]:
def validate_paths_from_questions_answer_pairs_in_corpus(df_qa_pairs: pd.DataFrame, document_paths: list):
  """
  df_qa_pairs = the Question and Answer Pairs from each data set all combined.
  document_paths = all the paths collected from the dataset .txt files
  """
  file_paths_matched = 0
  file_paths_not_matched = 0
  for index, qa_file_path in df_qa_pairs["ArticleFilesCustom"].items():
    if qa_file_path in document_paths:
      file_paths_matched += 1
    elif "," in qa_file_path:
      sub_file_paths_list = qa_file_path.split(",")
      for qa_sub_file_path in sub_file_paths_list:
        if qa_sub_file_path in document_paths:
          pass
        else:
          file_paths_not_matched += 1
          print(f"File not found: \n{qa_file_path}")
      file_paths_matched += 1
    else:
      file_paths_not_matched += 1
      print(f"File not found: \n{qa_file_path}")

  print(f"{file_paths_matched} Paths Found \n{file_paths_not_matched} Paths Not Found")
  return



In [ ]:
validate_paths_from_questions_answer_pairs_in_corpus(df_qa_pairs, document_paths)

3111 Paths Found 
0 Paths Not Found


## Generate Scores for Matching Documents

Optionally load the saved dataframe that has already been run the custom information retrieval system

In [ ]:
# import pandas as pd

# df_tfidf = pd.read_pickle('220927_df_tfidf.pickle')

## Create Metrics

Need to calculate:
* Success
* Failure
* Recall
* Precision
* F1



In [ ]:
actual_sample = """/content/Question_Answer_Dataset_v1.2/S08/data/set4/a1.txt,\
/content/Question_Answer_Dataset_v1.2/S08/data/set4/a4.txt"""

predicted_sample = """/content/Question_Answer_Dataset_v1.2/S08/data/set4/a9.txt,\
/content/Question_Answer_Dataset_v1.2/S08/data/set4/a8.txt,\
/content/Question_Answer_Dataset_v1.2/S08/data/set4/a7.txt,\
/content/Question_Answer_Dataset_v1.2/S08/data/set4/a4.txt,\
/content/Question_Answer_Dataset_v1.2/S08/data/set4/a1.txt"""


In [ ]:
def get_true_false_pos_neg(actual: str, predicted: str, corpus_lenght: int = 150 ):
  """
	True Positive (TP) - actual = 1, predicted = 1
	False Positive (FP) - actual = 0, predicted = 1
	False Negative (FN) - actual = 1, predicted = 0
	True Negative (TN) - actual = 0, predicted = 0
	"""
  # Conver the comma separated strings into lists
  actual = actual.split(",")
  predicted = predicted.split(",")

  # Define the true positives as the numeber of items that are in both the actual
  # and the predicted paths
  tp = len(set(actual).intersection(predicted))

  # Define false positves as the total number of predictions minus what was correct
  fp = len(set(predicted)) - tp

  # Define the false negatives as 0 since we are not explictle guessing the negative class
  fn = 0

  # Define the true negatives as the length of the corpus minus the number of guesses
  # Rather than pass in the lenght of the 
  tn = corpus_lenght - len(predicted)

  return tp, fp, fn, tn

In [ ]:
tp_sample, fp_sample, fn_sample, tn_sample = get_true_false_pos_neg(actual = actual_sample, predicted = predicted_sample)
print(tp_sample, fp_sample, fn_sample, tn_sample)

2 3 0 145


Calculate success of predicted documents

In [ ]:
def calculate_success(actual: str, predicted: str):
  # Calculate if the documnet path answers were correctly predicted
  # Compare the number of the correctly predicted answers to the number of correct
  # answers provided for a given query

  # Conver the comma separated strings into lists
  actual = actual.split(",")
  predicted = predicted.split(",")

  success = 0
  fail = 0

  if len(set(actual).intersection(predicted)) == len(actual):
    success = 1
  else:
    fail = 1

  return success, fail

In [ ]:
success_sample, fail_sample = calculate_success(actual = actual_sample, predicted = predicted_sample)
success_sample, fail_sample

(1, 0)

Calculate precision

In [ ]:
def calculate_precision(tp: int, fp: int):
  # Precision = TP / (TP + FP)
  precision = tp / (tp + fp)
  return precision

In [ ]:
precision_sample = calculate_precision(tp = tp_sample, fp = fp_sample)
precision_sample

0.4

Calculate recall

In [ ]:
def calculate_recall(tp: int, fn: int):
  # Recall = TP / (TP + FN)
  try:
    recall = tp / (tp + fn)
  except:
    recall = 0.0
  return recall

In [ ]:
recall_sample = calculate_recall(tp = tp_sample, fn = fn_sample)
recall_sample

1.0

Calculate F1

In [ ]:
def calculate_f1(precision: float, recall :float):
  # Calculate the F1 score where F1 = 2* (precision * recall) / (precision + recall)
  try:
    f1 = 2* (precision * recall) / (precision + recall)
  except:
    f1 = 0.0
  return f1

In [ ]:
f1_sample = calculate_f1(precision = precision_sample, recall = recall_sample)
f1_sample

0.5714285714285715

## Apply Metrics

Next, apply the custom metrics to the out put of the information retrieval system

In [ ]:
df_qa_pairs.columns

Index(['Dataset', 'ArticleTitle', 'Question', 'Answer',
       'DifficultyFromQuestioner', 'DifficultyFromAnswerer', 'ArticleFile',
       'ArticleFilesCustom'],
      dtype='object')

In [ ]:
def apply_custom_metrics_to_df(df: pd.DataFrame, actuals_col_name: str = 'ArticleFilesCustom', predicted_col_name: str = 'MatchingDocuments'):
  # Make a copy of the input dataframe
  df_original = df.copy()

  # Calculate True and False Positves and Negatives
  df_true_false_pos_neg = df_original.apply(lambda row: get_true_false_pos_neg(actual = row[actuals_col_name], predicted = row[predicted_col_name]), axis='columns', result_type='expand')
  df_true_false_pos_neg.columns = ['tp', 'fp', 'fn', 'tn']
  df_metrics = pd.concat([df_original, df_true_false_pos_neg], axis='columns')

  # Calculate Success
  df_success = df_metrics.apply(lambda row: calculate_success(actual = row[actuals_col_name], predicted = row[predicted_col_name]), axis='columns', result_type='expand')
  df_success.columns = ['success', 'fail']
  df_metrics = pd.concat([df_metrics, df_success], axis='columns')

  # Calculate Precision
  df_precision = df_metrics.apply(lambda row: calculate_precision(tp = row['tp'], fp = row['fp']), axis='columns', result_type='expand')
  df_metrics['precision'] = df_precision

  # Calculate Recall
  df_recall = df_metrics.apply(lambda row: calculate_recall(tp = row['tp'], fn = row['fn']), axis='columns', result_type='expand')
  df_metrics['recall'] = df_recall

  # Calculate F1
  df_f1 = df_metrics.apply(lambda row: calculate_f1(precision = row['precision'], recall = row['recall']), axis='columns', result_type='expand')
  df_metrics['f1'] = df_f1

  return df_metrics


Apply the custom metrics to the Information Retrieval System Output

In [ ]:
df_tfidf_with_metrics = apply_custom_metrics_to_df(df = df_tfidf, actuals_col_name = 'ArticleFilesCustom', predicted_col_name = 'MatchingDocuments')
df_tfidf_with_metrics.head(3)

,Dataset,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile,ArticleFilesCustom,MatchingDocuments,tp,fp,fn,tn,success,fail,precision,recall,f1
0,S08,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,data/set3/a4,/content/Question_Answer_Dataset_v1.2/S08/data...,/content/Question_Answer_Dataset_v1.2/S08/data...,1,149,0,0,1,0,0.006667,1.0,0.013245
1,S08,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,easy,easy,data/set3/a4,/content/Question_Answer_Dataset_v1.2/S08/data...,/content/Question_Answer_Dataset_v1.2/S08/data...,1,149,0,0,1,0,0.006667,1.0,0.013245
2,S08,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,easy,medium,data/set3/a4,/content/Question_Answer_Dataset_v1.2/S08/data...,/content/Question_Answer_Dataset_v1.2/S08/data...,1,149,0,0,1,0,0.006667,1.0,0.013245


In [ ]:
# df_tfidf_with_metrics.to_csv('221005_df_tfidf_with_metrics.csv')

Calculate the summary metrics

In [ ]:
def summary_metrics(df_with_metrics: pd.DataFrame):
  df = df_with_metrics.copy()

  # Success
  success = df['success'].sum()

  # Fail
  fail = df['fail'].sum()

  # Precision
  precision = df['precision'].mean()

  # Recall
  recall = df['recall'].mean()

  # F1
  f1 = df['f1'].mean()

  return success, fail, precision, recall, f1


In [ ]:
summary_metrics(df_with_metrics = df_tfidf_with_metrics)

(3111, 0, 0.006694524804457302, 1.0, 0.013299652152343092)

TODO:

Create formatted output that passes the grading criteria

```
Success | # | failure | # 
Recall | # 
Precision | # 
F1 | #
```



In [ ]:
def create_metrics_output_file(success: float, fail: float, precision: float, recall: float, f1: float) -> str:
  content = f"""Success|{success}|Failure|{fail}\navg recall|{recall}\navg precision|{precision}\navg F1|{f1}"""
  return content

In [ ]:
metrics_message = create_metrics_output_file(*summary_metrics(df_with_metrics = df_tfidf_with_metrics))

In [ ]:
# Open the file in the write mode
with open('test_csv_write.csv', 'w') as f:
    # create the csv writer
    print(metrics_message)
    f.write(metrics_message)

Success|3111|Failure|0
avg recall|1.0
avg precision|0.006694524804457302
avg F1|0.013299652152343092


Run the validator to endure file format works

In [ ]:
import csv
import sys
from os.path import exists
from pathlib import Path

metrics_file_path = "/content/test_csv_write.csv"

with open(metrics_file_path, newline='\n') as csvfile: #sys.argv[1]
    spamreader = csv.reader(csvfile, delimiter='|', quotechar='"')
    inQuestion = False
    succFailLabels = False
    hasRecall = False
    hasPrec = False
    hasF1 = False
    count = 0
    for row in spamreader:
      if (not row and count > 0):
        print("Error: row ", count, " is empty")
      if (len(row) == 1):
        inQuestion = True
      if (len(row) == 2 and row[0] == ""):
        if (not exists(Path(row[1].strip()))):
            print("Error: file ", row[1], " does not exist")
      if (len(row) == 4):
        if (row[0].strip() != "Success" or row[2].strip() != "Failure"):
            print("bad labels in ", row)
        else:
            succFailLabels = True
      if (len(row) == 2 and row[0].startswith("avg")):
        if (row[0].strip() == "avg recall"):
            hasRecall = True
        elif (row[0].strip() == "avg precision"):
            hasPrec = True
        elif (row[0].strip() == "avg F1"):
            hasF1 = True
        else:
            print("bad row: ", row)
            row = row + 1
    if (not hasRecall or not hasPrec or not hasF1):
      print("Missing metrics")
    else:
      print("Format all good")

Format all good


In [ ]:
# 

# TODO: This Part

TODO:
* Figure out a format for combining multiple correct files into one line
* Figure out how to conver the 5 predicted paths and find out the set() between them

# Reference

Good Kaggle areticle for the Question and Answer part.

[https://www.kaggle.com/code/francispimentel/finding-the-closest-question-and-then-anwering-it](https://www.kaggle.com/code/francispimentel/finding-the-closest-question-and-then-anwering-it)

See tutorial on Information Retreval [Information Retrieval](https://github.com/LearnPythonWithRune/MachineLearningWithPython/blob/main/colab/final/13%20-%20Project%20-%20Information%20Retrieval%20(IR).ipynb)

Youtube Tutorial on Information Retrieval [Video](https://www.youtube.com/watch?v=OUYioG0cydI&ab_channel=LearnPythonwithRune)

Nice example of TF-IDF and cosine similarity [Article](https://courses.cs.washington.edu/courses/cse373/17au/project3/project3-2.html)